In [2]:
import numpy as np
import os
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
def kl_hist(a,b,min_idx=1,smooth=1e-9):
    res = 0
    a=a[min_idx:]+smooth
    b=b[min_idx:]+smooth
    a/=np.linalg.norm(a)
    b/=np.linalg.norm(b)
    for i,j in zip(a,b):
        res+= i*np.log(i/j)
        res+= j*np.log(j/i)
    return res

In [28]:
def tes_func(x):
    bins = np.array([0, 13.86693361, 15.63797682, 17.40902002, 19.18006322,
        20.95110643, 22.72214963, 24.49319283, 26.26423604, 28.03527924,
        29.80632244, 31.57736565, 33.34840885, 35.11945205, 36.89049526,39,
        50])

    x = x.flatten()
    year = 2030
    text = open('../gm-games/src/worker/core/gs2.txt','rt').read()
    #x = abs(x)
    print(x,end='\t')
   # ['0.0595529939813699', '0.6729315722656376', '0.0547439860604122', '1.0', '0.0887886919990636'
    if True:
        s1 = str(np.clip(x[0],0,1))
        s2 = str(np.clip( x[1],0,1))
        s3 = str(np.clip( x[2],0,1))
        s5 = str(np.clip( x[3],0,1))
    else:
        s1 = str(x[0])
        s2 = str(x[1])
        s3 = str(x[2])
        s5 = str(x[3])
    
    text2 = text
    subs = [s1,s2,s3,s5]
    for i in range(len(subs)):
        text2 = text2.replace('STR_{}'.format(i+1),subs[i])
    with open('../gm-games/src/worker/core/GameSim.basketball.ts','wt') as fp:
        fp.write(text2)
    time.sleep(5)



    driver = webdriver.Chrome(executable_path='./chromedriver.exe')

    driver.get("http://localhost:3000/new_league/random")
    dash_name = 'btn btn-primary dashboard-top-link dashboard-top-link-new mr-sm-3 mb-3'
    delay = 5 # seconds
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, 'actual-actual-content')))
    except TimeoutException:
        print("Loading took too much time!")
        raise
    except:
        raise
    element = driver.find_element(By.XPATH, '//button[text()="Create League"]')
    ActionChains(driver).click(element).perform()

    delay = 15 # seconds
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[1]/div[6]/div[1]/div/div[1]/h2')))
    except TimeoutException:
        print("Loading took too much time!")
        raise
    except:
        raise
    driver.set_window_size(1920, 1200)

    element = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[5]/div/ul[7]/li[8]/a')
    ActionChains(driver).move_to_element(element).click(element).perform()
    delay = 5 # seconds
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[1]/div[6]/div[1]/div/div[2]/div/a[15]/h3')))
    except TimeoutException:
        print("Loading took too much time!")
        raise
    time.sleep(0.3)
    
        
    element = driver.find_element_by_partial_link_text('Auto Play')
    ActionChains(driver).move_to_element(element).click(element).perform()
    delay = 15 # seconds
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div/div/div[2]/button[2]')))
    except TimeoutException:
        print("Loading took too much time!")
        raise

    time.sleep(0.3)

    
    
    inputField = driver.find_element_by_xpath('/html/body/div[4]/div/div/div[1]/form/div/div[1]/input')

    inputField.clear()
    inputField.send_keys(str(year))


    element = driver.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/button[2]')
    ActionChains(driver).click(element).perform()
    time.sleep(2)

    element = driver.find_element_by_xpath('//*[@id="content"]/div[1]/button')
    ActionChains(driver).move_to_element(element).click(element).perform()
    
    delay = 2800 # seconds
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, "//span[text()='{} regular season']".format(year))))
    except TimeoutException:
        print("Loading took too much time!")
        raise
    except:
        raise
    fn = '../../Downloads/BBGM_League_1_all_seasons_Average_Stats.csv'
    if os.path.exists(fn):
        os.remove(fn)
        
    element = driver.find_element_by_partial_link_text('Export Stats')
    ActionChains(driver).move_to_element(element).click(element).perform()
    delay = 15 # seconds
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[2]/div[6]/div[1]/form/button')))
    except TimeoutException:
        print("Loading took too much time!")
        raise
    except:
        raise
    element = driver.find_element_by_xpath('/html/body/div/div[2]/div[6]/div[1]/form/button')
    ActionChains(driver).click(element).perform()

    time.sleep(2)
    import pandas as pd

    df = pd.read_csv(fn)
    
    df_nba = pd.read_csv('nba_min_recent.csv',index_col=0)
    df_nba['MP'] = df_nba.mp/df_nba.g
    dfv = df[df.MP*df.G >= 850]
    dfv2 = df_nba[df_nba.mp >= 850]
    #sim_m,sim_s = dfv.MP.mean(),dfv.MP.var()
    #nba_m,nba_s =  dfv2.MP.mean(),dfv2.MP.var()
    #def kl(m1,s1,m2,s2):
    #    return np.log(np.sqrt(s2/s1)) + (s1 + (m1-m2)**2)/(2*s2) - 0.5
    
    hist1 = np.histogram(dfv.MP,bins,density=True)[0]
    hist2 = np.histogram(dfv2.MP,bins,density=True)[0]
    err = kl_hist(hist1,hist2)
    driver.close()
    print(err)
    return err
    #return kl(sim_m,sim_s,nba_m,nba_s) + kl(nba_m,nba_s,sim_m,sim_s)

In [44]:

x = np.array([0.016,0.728,0.051,0.094])
#array([0.0125926261330508, 0.7277571249131707, 0.0508749297562149,
#       0.0955069419788015])
#x = np.zeros_like(x)
#x[-1] = 1
tes_func(x)

[0.016 0.728 0.051 0.094]	0.283172185164157


0.283172185164157

In [35]:
if False:
    import GPy
    import GPyOpt
    bounds = [{'name': 'min', 'type': 'continuous', 'domain': (0,0.1)},
              {'name': 'sub', 'type': 'continuous', 'domain': (0.6,0.8)},
              {'name': '+', 'type': 'continuous', 'domain': (0.04,0.15)},
              {'name': '-', 'type': 'continuous', 'domain': (0.04,0.15)},
             ]
    myProblem = GPyOpt.methods.BayesianOptimization(tes_func,bounds,
                                                    model_type='GP',
                                                    verbosity=True,
                                                    verbose=True,
                                                    verbose_model=True,
                                                    acquisition_type='EI',
                                                    exact_feval = False,
                                                    normalize_Y=True, 
                    batch_size=1, num_cores=1,
                    verbosity_model=True, de_duplication=True)

    myProblem.run_optimization(100000,60*60*2, verbosity=True)
    print(myProblem.fx_opt)



In [36]:
if False:
    x,y = myProblem.get_evaluations()
    x[:-1,:].shape,y.shape
    x[np.argmin(y)]

In [ ]:
import cma #
#[0.0207189191227875 0.7293522577303776 0.0521512661333346
# 0.0897818188873167]	.20
#[0.0159008527131689 0.7276145375540903 0.0513179682625608
# 0.0939385899349409]	
# 

In [ ]:
es = cma.CMAEvolutionStrategy(x, 0.005, {'tolx':1e-3,'tolfun':1e-3,'maxfevals':1000})
es.optimize(tes_func,n_jobs=0)

In [40]:
#['0.06572070602774815', '0.6729067959843927', '0.05780716720676334', '1.0', '0.0988443077002651',  # seemed good
es.result_pretty()  #

np.set_printoptions(suppress=False)
np.set_printoptions(precision=16)
abs(es.best.x)
es.mean

final/bestever f-value = 2.165282e-01 2.006579e-01
incumbent solution: [0.012592626133050781, 0.7277571249131707, 0.050874929756214916, 0.09550694197880147]
std deviation: [0.001549985782276617, 0.000853819541255777, 0.0007635354980028928, 0.001237718087429697]


array([0.0125926261330508, 0.7277571249131707, 0.0508749297562149,
       0.0955069419788015])

In [41]:
es.best.x

array([0.0159008527131689, 0.7276145375540903, 0.0513179682625608,
       0.0939385899349409])

In [ ]:
plt.hist(df_nba['mp']/df_nba['g'],40)

In [ ]:
(df_nba['mp']).min()

In [ ]:
import matplotlib.pyplot as plt
plt.hist(df_nba['mp']/df_nba['g'],40,density=True,label='NBA')
_ = plt.hist(.MP,40,density=True,label='BBGM')


In [ ]:
sim_m,sim_s,nba_m,nba_s

In [ ]:
np.array(nba_m).shape